In [1]:
import jax
import jax.tree_util as jtu
import jax.numpy as jnp
import functools
from dataclasses import dataclass
import numpy as np

import genjax
from genjax.typing import Callable, Any

key = jax.random.PRNGKey(314159)
console = genjax.pretty()


_global = jnp.arange(3)


@genjax.gen
def localization_kernel(x):
    y = genjax.normal(jnp.sum(_global), 1.0) @ "x"
    return x + y


def wrap(fn):
    @genjax.gen
    def inner(carry, *static_args):
        idx, state = carry
        newstate = fn.inline(state, *static_args)
        return idx + 1, newstate

    return inner


localization_chain = genjax.Unfold(wrap(localization_kernel), max_length=3)

_, trace = genjax.simulate(localization_chain)(key, (2, (0, 0.0)))

In [2]:
trace

VectorTrace
├── gen_fn
│   └── UnfoldCombinator
│       ├── max_length
│       │   └── (const) 3
│       └── kernel
│           └── BuiltinGenerativeFunction
│               └── source
│                   └── <function inner>
├── indices
│   └──  i32[3]
├── inner
│   └── BuiltinTrace
│       ├── gen_fn
│       │   └── BuiltinGenerativeFunction
│       │       └── source
│       │           └── <function inner>
│       ├── args
│       │   └── tuple
│       │       └── tuple
│       │           ├──  i32[3]
│       │           └──  f32[3]
│       ├── retval
│       │   └── tuple
│       │       ├──  i32[3]
│       │       └──  f32[3]
│       ├── choices
│       │   └── Trie
│       │       └── :x
│       │           └── DistributionTrace
│       │               ├── gen_fn
│       │               │   └── Normal
│       │               ├── args
│       │               │   └── tuple
│       │               │       ├──  i32[3]
│       │               │       └──  f32[3]
│       │               ├── value
│       │               │   └──  f32[3]
│       │               └── score
│       │                   └──  f32[3]
│       ├── cache
│       │   └── Trie
│       └── score
│           └──  f32[3]
├── args
│   └── tuple
│       ├── (const) 2
│       └── tuple
│           ├── (const) 0
│           └── (const) 0.0
├── retval
│   └── tuple
│       ├──  i32[3]
│       └──  f32[3]
└── score
    └──  f32[]